In [ ]:
import pandas as pd

from summer.utils import ref_times_to_dti

from autumn.core import inputs
from autumn.core.utils.display import pretty_print
from autumn.settings.constants import COVID_BASE_DATETIME
from autumn.core.project import get_project
from autumn.projects.sm_sir.australia.northern_territory.project import get_ts_date_indexes

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
region = "northern_territory"
p = get_project("sm_sir", region, reload=True)
baseline_params = p.param_set.baseline
model = p.run_baseline_model(baseline_params)
derived_df = model.get_derived_outputs_df()
ts_set_dates = get_ts_date_indexes(p.ts_set, COVID_BASE_DATETIME)

In [ ]:
pretty_print(baseline_params)

In [ ]:
def get_comparison_df(indicator):
    return pd.DataFrame({
        "modelled": derived_df[indicator],
        "data": ts_set_dates[indicator]
    })

In [ ]:
get_comparison_df("notifications").plot()

In [ ]:
get_comparison_df("hospital_admissions").plot()

In [ ]:
# get_comparison_df("icu_admissions").plot()

In [ ]:
# get_comparison_df("infection_deaths").plot()

In [ ]:
sc_models = p.run_scenario_models(
    model, p.param_set.scenarios,
)

In [ ]:
indicator = "notifications"
chosen_scenario = 1  # Scenario number minus one
sc_results = sc_models[chosen_scenario].get_derived_outputs_df()
pd.DataFrame(
    {
        0: derived_df[indicator],
        1: sc_results[indicator],
    }
).plot()

In [ ]:
# Check population distribution by vaccination status
sc_results[[f"prop_immune_{strat}" for strat in model._stratifications[3].strata]].plot.area()

In [ ]:
def get_indigenous_prop(results, indicator):
    indigenous_values = results[f"{indicator}Xindigenous"].sum()
    non_indigenous_values = results[f"{indicator}Xnon_indigenous"].sum()
    return indigenous_values / (indigenous_values + non_indigenous_values)

In [ ]:
prop_modelled_hosp_indigenous = get_indigenous_prop(derived_df, 'hospital_admissions')
prop_actual_hosp_indigenous = 24091. / (24091 + 69259)
ratio = prop_modelled_hosp_indigenous / prop_actual_hosp_indigenous
print(f"Modelled proportion of hospital admissions Indigenous: {prop_modelled_hosp_indigenous}")
print(f"Actual proportion of hospital admissions: {prop_actual_hosp_indigenous}")
print(f"Model is out by a factor of: {ratio}")
1. / ratio

In [ ]:
prop_modelled_icu_indigenous = get_indigenous_prop(derived_df, 'icu_admissions')
prop_actual_icu_indigenous = 46. / (20. + 46.)
ratio = prop_modelled_icu_indigenous / prop_actual_icu_indigenous
print(f"Modelled proportion of ICU admissions Indigenous: {prop_modelled_icu_indigenous}")
print(f"Actual proportion of ICU admissions: {prop_actual_icu_indigenous}")
print(f"Model is out by a factor of: {ratio}")
5. / ratio

In [ ]:
prop_modelled_deaths_indigenous = get_indigenous_prop(derived_df, 'infection_deaths')
prop_actual_deaths_indigenous = 28. / (28. + 32.)
ratio = prop_modelled_deaths_indigenous / prop_actual_deaths_indigenous
print(f"Modelled proportion of deaths Indigenous: {prop_modelled_deaths_indigenous}")
print(f"Actual proportion of deaths: {prop_actual_deaths_indigenous}")
print(f"Model is out by a factor of: {ratio}")
2.94 / ratio

In [ ]:
def get_cumu_comparison_df(indicator):
    return pd.DataFrame({
        "modelled": derived_df[f"cumulative_{indicator}"],
        "data": ts_set_dates[indicator].cumsum()
    })

In [ ]:
get_cumu_comparison_df("icu_admissions").plot()

In [ ]:
get_cumu_comparison_df("infection_deaths").plot()